In [7]:
import sqlite3

def polaczZBaza():
    return sqlite3.connect('logs_database.db')

def pobierzWszystkoZTabeli(tabela):
    zapytanieSchemat = f"PRAGMA table_info({tabela})"
    zapytanieDane = f"SELECT * FROM {tabela}"
    with polaczZBaza() as baza:
        kursor = baza.cursor()
        nazwyKolumn = [kolumna[1] for kolumna in kursor.execute(zapytanieSchemat)]
        daneTabeli = [wiersz for wiersz in kursor.execute(zapytanieDane)]
        return nazwyKolumn, daneTabeli

nazwyKolumnZNginx, daneZNginx = pobierzWszystkoZTabeli("nginx_logs")
nazwyKolumnZErrorApache, daneZErrorApache = pobierzWszystkoZTabeli("apache_error_logs")
nazwyKolumnZAccessApache, daneZAccessApache = pobierzWszystkoZTabeli("apache_access_logs")
nazwyKolumn = [["nginx", nazwyKolumnZNginx], ["apache_error_logs", nazwyKolumnZErrorApache], ["apache_access_logs", nazwyKolumnZAccessApache]]
daneZTabel = [["nginx", daneZNginx], ["apache_error_logs", daneZErrorApache], ["apache_access_logs", daneZAccessApache]]

print(nazwyKolumn)
print(daneZTabel)

# nazwyKolumnZNginx, daneZNginx = pobierzWszystkoZTabeli("apache_error_logs")
# print(nazwyKolumnZNginx)
# print(daneZNginx)
# print(pobierzWszystkoZTabeli("nginx_logs"))
# print(pobierzWszystkoZTabeli("apache_access_logs"))
# print(pobierzWszystkoZTabeli("apache_error_logs"))

[['nginx', ['id', 'ip', 'date_time', 'method', 'url', 'protocol', 'status', 'size', 'user_agent']], ['apache_error_logs', ['id', 'date_time', 'log_level', 'pid', 'message']], ['apache_access_logs', ['id', 'ip', 'date_time', 'method', 'url', 'protocol', 'status', 'size']]]
[['nginx', [(1, '107.195.138.129', '01/Dec/2024:16:57:30 +0000', 'PATCH', '/dashboard', 'HTTP/1.1', 400, 2458, 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.164 Safari/537.36'), (2, '234.24.237.153', '01/Dec/2024:11:53:44 +0000', 'PATCH', '/static/js/app.js', 'HTTP/1.1', 401, 2730, 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36'), (3, '160.118.184.9', '28/Nov/2024:11:37:40 +0000', 'PATCH', '/images/logo.png', 'HTTP/1.2', 200, 3678, 'Mozilla/5.0 (iPhone; CPU iPhone OS 15_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.0 Mobile/15E148 Safari/604.1'), (4, '42.188.253.131', '

In [8]:
def polaczZBaza():
    return sqlite3.connect('logs_database.db')

def pobierz_wszystko_z_tabeli(tabela):
    zapytanie_schemat = f"PRAGMA table_info({tabela})"
    zapytanie_dane = f"SELECT * FROM {tabela}"
    with polaczZBaza() as baza:
        kursor = baza.cursor()
        # Get column names
        nazwy_kolumn = [kolumna[1] for kolumna in kursor.execute(zapytanie_schemat)]
        # Get table data
        dane_tabeli = kursor.execute(zapytanie_dane).fetchall()
        return nazwy_kolumn, dane_tabeli

# List of table names to process
tabele = ["nginx_logs", "apache_error_logs", "apache_access_logs"]

# Retrieve column names and data for each table
wyniki = [
    {
        "tabela": tabela,
        "kolumny": pobierz_wszystko_z_tabeli(tabela)[0],
        "dane": pobierz_wszystko_z_tabeli(tabela)[1],
    }
    for tabela in tabele
]

# Example of how to access the results
for wynik in wyniki:
    print(f"Tabela: {wynik['tabela']}")
    print(f"Kolumny: {wynik['kolumny']}")
    print(f"Dane: {wynik['dane']}")

Tabela: nginx_logs
Kolumny: ['id', 'ip', 'date_time', 'method', 'url', 'protocol', 'status', 'size', 'user_agent']
Dane: [(1, '107.195.138.129', '01/Dec/2024:16:57:30 +0000', 'PATCH', '/dashboard', 'HTTP/1.1', 400, 2458, 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.164 Safari/537.36'), (2, '234.24.237.153', '01/Dec/2024:11:53:44 +0000', 'PATCH', '/static/js/app.js', 'HTTP/1.1', 401, 2730, 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36'), (3, '160.118.184.9', '28/Nov/2024:11:37:40 +0000', 'PATCH', '/images/logo.png', 'HTTP/1.2', 200, 3678, 'Mozilla/5.0 (iPhone; CPU iPhone OS 15_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.0 Mobile/15E148 Safari/604.1'), (4, '42.188.253.131', '01/Dec/2024:05:51:40 +0000', 'PUT', '/profile', 'HTTP/1.2', 200, 3553, 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:94.0) Gecko/20100101 Firefox/94.0'), (5, '81.110